# Proyecto de Reconocimiento Facial con Flask

## Introducción
El reconocimiento facial se ha convertido en una tecnología crucial en diversos campos, como la seguridad, la autenticación de usuarios y la gestión de identidades. Este proyecto tiene como objetivo desarrollar una aplicación de reconocimiento facial utilizando el framework Flask, que permita identificar y verificar rostros de manera eficiente y precisa. La aplicación está optimizada para su uso en distribuciones Linux y fue diseñada inicialmente para ejecutarse en una Raspberry Pi.

## Objetivos
**Objetivo General:**
Desarrollar una aplicación web de reconocimiento facial utilizando Flask.
**Objetivos Específicos:**
- Implementar una API RESTful para la gestión de rostros.
- Integrar bibliotecas de reconocimiento facial.
- Crear una interfaz web para la carga y visualización de imágenes.
- Optimizar el desarrollo para distribuciones Linux y Raspberry Pi.
- Utilizar el entorno Anaconda, Spyder, PyChart y Visual Studio Code para el desarrollo.

## Metodología
El proyecto se desarrolló utilizando la metodología ágil, con iteraciones cortas y entregas frecuentes. Las principales herramientas utilizadas fueron Python, Flask, OpenCV, face_recognition, y los entornos de desarrollo Anaconda, Spyder, PyChart y Visual Studio Code. Se realizaron optimizaciones específicas para garantizar un rendimiento eficiente en distribuciones Linux y en la Raspberry Pi.

### Preparación del Espacio de Trabajo

#### Software Utilizado

- Anaconda: Una distribución de Python que incluye una gran cantidad de paquetes científicos y de desarrollo. Anaconda facilita la gestión de entornos y dependencias, asegurando que todas las bibliotecas necesarias estén disponibles y actualizadas.

- Spyder: Un entorno de desarrollo integrado (IDE) que viene con Anaconda. Spyder ofrece un entorno interactivo para el desarrollo de scripts en Python, con herramientas avanzadas para la depuración y el análisis de código.

- PyCharm: Utilizado para la edición y gestión avanzada del código Python.

- Jupyter Notebooks: Herramienta utilizada para documentar el desarrollo del proyecto, proporcionando una estructura clara y accesible de la información técnica.

- Visual Studio Code: Un editor de código fuente ligero y potente, utilizado para la edición de scripts y archivos de configuración.

- Terminal: Utilizada para ejecutar comandos, gestionar entornos virtuales y controlar el flujo de trabajo de desarrollo


### Instalación y Configuración de Anaconda

**Descarga e instalación de Anaconda:**

- Descarga el instalador de Anaconda desde la página oficial.
- Ejecuta el instalador y sigue las instrucciones para completar la instalación.


**Creación de un entorno virtual:**

## Desarrollo del Proyecto
### Estructura del Proyecto
```
tu_proyecto/
├── app/
│   ├── __init__.py
│   ├── routes.py
│   ├── static/
│   │   ├── css/
│   │   │   └── styles.css
│   │   ├── js/
│   │   │   └── script.js
│   └── templates/
│       ├── index.html
│       └── success.html
├── models/
├── utils/
├── tests/
├── requirements.txt
├── run.py
└── README.md

```

### Principales Componentes
- **API RESTful:** Implementada con Flask para gestionar las solicitudes de reconocimiento facial.
- **Reconocimiento Facial:** Utilización de OpenCV y face_recognition para la detección y análisis de rostros.
- **Interfaz Web:** Desarrollo de una interfaz sencilla para la carga y visualización de imágenes.

In [ ]:
conda create --name reconocimiento_facial python=3.8
conda activate reconocimiento_facial


### Código del Proyecto
#### Código Python (app.py)


In [1]:
import os
import datetime
import base64
import numpy as np
from flask import Flask, jsonify, request, render_template, redirect, url_for
import face_recognition
from io import BytesIO
from PIL import Image
import sqlite3

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = os.path.join(os.getcwd(), 'static', 'uploads')
DATABASE = 'database.db'


# Crear la base de datos y la tabla de usuarios si no existen
def init_db():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY, name TEXT, encoding BLOB)''')
    conn.commit()
    conn.close()


# Ejecutar la inicialización de la base de datos
init_db()


def decode_image(data_url):
    try:
        header, encoded = data_url.split(",", 1)
        data = base64.b64decode(encoded)
        image = Image.open(BytesIO(data))
        return image
    except Exception as e:
        print(f"Error decoding image: {e}")
        return None


def save_image(image, filename):
    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        image.save(filepath)
        return filepath
    except Exception as e:
        print(f"Error saving image: {e}")
        return None


def save_encoding(name, encoding):
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute("INSERT INTO users (name, encoding) VALUES (?, ?)", (name, encoding.tobytes()))
    conn.commit()
    conn.close()


def load_encodings():
    conn = sqlite3.connect(DATABASE)
    c = conn.cursor()
    c.execute("SELECT name, encoding FROM users")
    users = c.fetchall()
    conn.close()
    return {name: np.frombuffer(enc, dtype=np.float64) for name, enc in users}


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/register', methods=['POST'])
def register():
    data = request.json
    name = data.get('name')
    photo_data_url = data.get('photo')
    if photo_data_url and name:
        if not os.path.exists(app.config['UPLOAD_FOLDER']):
            os.makedirs(app.config['UPLOAD_FOLDER'])

        image = decode_image(photo_data_url)
        if image:
            filename = f"{datetime.date.today()}_{name}.jpg"
            filepath = save_image(image, filename)
            if filepath:
                encoding = face_recognition.face_encodings(face_recognition.load_image_file(filepath))
                if encoding:
                    save_encoding(name, encoding[0])
                    return jsonify({"message": "Registro exitoso"}), 200
    return jsonify({"message": "Error en el registro"}), 400


@app.route('/login', methods=['POST'])
def login():
    data = request.json
    photo_data_url = data.get('photo')
    if photo_data_url:
        if not os.path.exists(app.config['UPLOAD_FOLDER']):
            os.makedirs(app.config['UPLOAD_FOLDER'])

        image = decode_image(photo_data_url)
        if image:
            login_filename = os.path.join(app.config['UPLOAD_FOLDER'], "login_face.jpg")
            image.save(login_filename)

            login_image = face_recognition.load_image_file(login_filename)
            login_face_encodings = face_recognition.face_encodings(login_image)

            if len(login_face_encodings) > 0:
                login_face_encoding = login_face_encodings[0]
                registered_data = load_encodings()
                for name, registered_face_encoding in registered_data.items():
                    matches = face_recognition.compare_faces([registered_face_encoding], login_face_encoding)
                    if any(matches):
                        return jsonify({"success": True, "name": name})

        return jsonify({"success": False})
    return redirect(request.url)


@app.route('/success')
def success():
    user_name = request.args.get('user_name')
    return render_template('success.html', user_name=user_name)


if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Código HTML (index.html)


In [2]:
<!doctype html>
<html lang="en">

<head>
    <meta charset="utf-8" />
    <title>Ingenieria en software - Autentificacion Biometrico</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta content="Diseño de Sistema de Autenticación Biométrica con Reconocimiento Facial " name="description" />
    <meta content="Emmanuel_Campos" name="author" />

    <!-- favicon -->
    <link rel="shortcut icon" href="{{ url_for('static', filename='images/favicon.ico') }}">

    <!-- Font Awesome Icon -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css" integrity="sha512-iecdLmaskl7CVkqkXNQ/ZH/XLlvWZOJyj7Yy7tcenmpD1ypASozpmT/E0iPtmFIB46ZmdtAc9eNBvH0H/ZpiBw==" crossorigin="anonymous" referrerpolicy="no-referrer" />

    <!-- Style css -->
    <link href="{{ url_for('static', filename='css/style.min.css') }}" rel="stylesheet" type="text/css">
</head>

<body>
    <section class="h-screen flex items-center justify-center bg-no-repeat inset-0 bg-cover bg-[url('{{ url_for('static', filename='images/bg.png') }}')]">
        <div class="container 2xl:px-80 xl:px-52">
            <div class="bg-white rounded-lg p-5" style="box-shadow: rgba(99, 99, 99, 0.2) 0px 2px 8px 0px;">
                <div class="grid xl:grid-cols-5 lg:grid-cols-3 gap-6">

                    <div class="xl:col-span-2 lg:col-span-1 hidden lg:block">
                        <div class="bg-sky-600 text-white rounded-lg flex flex-col justify-between gap-10 h-full w-full p-7">
                            <span class="font-semibold tracking-widest uppercase">Ingeniería en Software</span>
                            <div>
                                <h1 class="text-3xl/tight mb-4">Proyecto de Reconocimiento Facial</h1>
                                <p class="text-gray-200 font-normal leading-relaxed">
                                    Este proyecto fue desarrollado como parte de la materia de Ingeniería en Software en la ESIME Zacatenco del Instituto Politécnico Nacional. 
                                    La aplicación permite a los usuarios registrarse y autenticarse utilizando sus rostros.
                                </p>
                            </div>
                            <div>
                                <div class="bg-sky-700/50 rounded-lg p-5">
                                    <p class="text-gray-200 text-sm font-normal leading mb-4">
                                        El proyecto está construido con Flask y JavaScript, utilizando la biblioteca <code>face_recognition</code> para el procesamiento de imágenes.
                                        <br><br>
                                        Desarrollado por: Emmanuel Campos Genaro<br>
                                        Número de Boleta: 2014300225<br>
                                        Escuela: ESIME Zacatenco, IPN<br>
                                        Prpfesora : Gabriela Montserrat Gonzalez Camacho 
                                    </p>

                                </div>
                            </div>
                        </div>
                    </div>

                    <div class="xl:col-span-3 lg:col-span-2 lg:mx-10 my-auto">
                        <div>
                            <h1 class="text-2xl/tight mb-3">Registro e Inicio de Sesión</h1>

                        </div>

                        <div class="space-y-5 mt-10">
                            <div>
                                <h2 class="text-xl font-semibold">Registro</h2>
                                <label class="font-medium text-sm block mb-2" for="name">Nombre</label>
                                <input class="text-gray-500 border-gray-300 focus:ring-0 focus:border-gray-400 text-sm rounded-lg py-2.5 px-4 w-full" type="text" id="name" name="name" placeholder="Ingrese su nombre">
                                <label class="font-medium text-sm block mt-4 mb-2" for="photo">Rostro</label>
                                <video id="video" width="440" height="280" autoplay class="border-gray-300 rounded-lg"></video>
                                <canvas id="canvas" width="440" height="280" style="display:none;"></canvas>

                            </div>
                            <div class="space-y-5 mt-10">
                                <div>
                                    <button onclick="register()" class="bg-sky-600 text-white text-sm rounded-lg px-6 py-2.5 mt-4">Registrar</button>
                                </div>
                            <div>
                                <h2 class="text-xl font-semibold mt-8">Inicio de Sesión</h2>

                                <button onclick="login()" class="bg-sky-600 text-white text-sm rounded-lg px-6 py-2.5 mt-4">Iniciar Sesión</button>
                            </div>
                        </div>
                    </div>

                </div>
            </div>
        </div>
    </section>

    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>

</html>


SyntaxError: invalid syntax (680882091.py, line 1)

#### Código HTML (success.html)


In [ ]:
<!doctype html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bienvenido</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.min.css') }}">
    <script src="https://cdn.tailwindcss.com"></script>
</head>

<body>
    <header class="bg-sky-600 text-white text-center py-4">
        <h1 class="text-3xl font-bold">Bienvenido, {{ user_name }}!</h1>
    </header>
    
    <main class="flex flex-col items-center justify-center h-screen bg-gray-100">
        <button onclick="logout()" class="bg-red-500 text-white text-lg rounded-lg px-6 py-2.5 mt-5">Cerrar Sesión</button>
    </main>
    
    <footer class="bg-gray-800 text-white text-center py-4">
        <p>&copy; 2024 Aplicación de Reconocimiento Facial</p>
    </footer>

    <script>
        function logout() {
            window.location.href = "/";
        }
    </script>
</body>

</html>


#### Código CSS (styles.css)


In [ ]:
body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
    display: flex;
    flex-direction: column;
    min-height: 100vh;
}

header, footer {
    background-color: #333;
    color: #fff;
    text-align: center;
    padding: 1em 0;
}

header h1, footer p {
    margin: 0;
}

main {
    flex: 1;
    padding: 1em;
}

#video-section, #register-section {
    margin: 1em 0;
}

#video {
    border: 1px solid #ccc;
    border-radius: 5px;
}

button {
    margin-top: 10px;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
}

button:hover {
    background-color: #f0f0f0;
}


#### Código JavaScript (script.js)


In [ ]:
const video = document.getElementById('video');
const canvas = document.getElementById('canvas');
const context = canvas.getContext('2d');

navigator.mediaDevices.getUserMedia({ video: true })
    .then(stream => {
        video.srcObject = stream;
    })
    .catch(error => {
        console.error("Error accessing the camera: ", error);
    });

function capture() {
    context.drawImage(video, 0, 0, canvas.width, canvas.height);
    return canvas.toDataURL('image/jpeg');
}

async function register() {
    const name = document.getElementById('name').value;
    const photo = capture();

    try {
        const response = await fetch('/register', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ name, photo })
        });
        const data = await response.json();
        if (data.message === "Registro exitoso") {
            alert('Registro exitoso');
        } else {
            alert('Error en el registro');
        }
    } catch (error) {
        console.error("Error:", error);
    }
}

async function login() {
    const photo = capture();

    try {
        const response = await fetch('/login', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ photo })
        });
        const data = await response.json();
        if (data.success) {
            window.location.href = `/success?user_name=${data.name}`;
        } else {
            alert('No se encontró ninguna coincidencia');
        }
    } catch (error) {
        console.error("Error:", error);
    }
}


## Resultados y Discusión
La aplicación desarrollada logró identificar y verificar rostros con una alta precisión. Las pruebas realizadas en distribuciones Linux y en la Raspberry Pi demostraron un rendimiento satisfactorio. Sin embargo, se identificaron áreas de mejora, como la optimización del rendimiento en condiciones de baja luz y la implementación de medidas adicionales de seguridad.

## Conclusiones
El proyecto demostró que es posible desarrollar una aplicación web de reconocimiento facial eficiente utilizando Flask y bibliotecas especializadas en Python. La optimización para Linux y Raspberry Pi permitió crear una solución portátil y de bajo costo. Futuras investigaciones podrían centrarse en mejorar la precisión del reconocimiento y en la integración de la aplicación con sistemas de seguridad más amplios.

## Referencias
- Jones, M. (2020). *Face Recognition with Python*. Journal of Computer Vision.
- OpenCV Documentation. (2021). Retrieved from https://docs.opencv.org/
- Flask Documentation. (2021). Retrieved from https://flask.palletsprojects.com/
- Bledsoe, W. W. (1966). The model method in facial recognition. Panoramic Research Inc.
- Viola, P., & Jones, M. (2001). Rapid object detection using a boosted cascade of simple features. Proceedings of the 2001 IEEE Computer Society Conference on Computer Vision and Pattern Recognition (CVPR), 1, 511-518.
- Turk, M., & Pentland, A. (1991). Eigenfaces for recognition. Journal of Cognitive Neuroscience, 3(1), 71-86.
- Belhumeur, P. N., Hespanha, J. P., & Kriegman, D. J. (1997). Eigenfaces vs. fisherfaces: Recognition using class specific linear projection. IEEE Transactions on Pattern Analysis and Machine Intelligence, 19(7), 711-720.
- Parkhi, O. M., Vedaldi, A., & Zisserman, A. (2015). Deep face recognition. BMVC.
- Schroff, F., Kalenichenko, D., & Philbin, J. (2015). FaceNet: A unified embedding for face recognition and clustering. Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, 815-823.
- Taigman, Y., Yang, M., Ranzato, M. A., & Wolf, L. (2014). DeepFace: Closing the gap to human-level performance in face verification. Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, 1701-1708.
- Dlib. (n.d.). Dlib C++ Library. Retrieved from http://dlib.net/
- OpenCV. (n.d.). OpenCV Documentation. Retrieved from https://docs.opencv.org/
- Flask. (n.d.). Flask Documentation. Retrieved from https://flask.palletsprojects.com/
- King, D. E. (2009). Dlib-ml: A machine learning toolkit. Journal of Machine Learning Research, 10, 1755-1758.
- Anaconda. (n.d.). Anaconda Distribution. Retrieved from https://www.anaconda.com/products/individual
- Python Software Foundation. (n.d.). Python Language Reference, version 3.8. Retrieved from https://www.python.org/
- GitHub. (n.d.). GitHub Documentation. Retrieved from https://docs.github.com/
